Autores:
- Hernán Francisco Díaz Farías
- María Angélica Parra Mendoza
- Loreto Constanza Zavala Gaete

# Tarea 1
### Realice una exploración de datos para ver que datos existen, cuales de ellos entregan información relevante para el problema y cuales no, eliminación (si corresponde) de datos nulos y duplicados, entre otros.

#### Librerías a utilizar

In [29]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
import matplotlib.pyplot as plt
#from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes

#### Cargar el archivo csv

In [30]:
# Cargar el archivo csv
df = (
    pd.read_csv(
        '../data/apartments_for_rent_classified_100K.csv',
        encoding='ISO-8859-1',
        sep=';',
        low_memory=False # Para evitar warnings de dtype
        )
    )

#### Visualización de los datos

In [31]:
df.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668640009,housing/rent/apartment,One BR 507 & 509 Esplanade,"This unit is located at 507 & 509 Esplanade, R...",NaN,1,1,USD,No,Thumbnail,...,"$2,20",Monthly,542,507 509 Esplanade,Redondo Beach,CA,338.520,-1.183.759,RentLingo,1.577360e+09
1,5668639818,housing/rent/apartment,Three BR 146 Lochview Drive,"This unit is located at 146 Lochview Drive, Ne...",NaN,1.5,3,USD,No,Thumbnail,...,"$1,25",Monthly,1500,146 Lochview Dr,Newport News,VA,370.867,-764.941,RentLingo,1.577360e+09
2,5668639686,housing/rent/apartment,Three BR 3101 Morningside Drive,This unit is located at 3101 Morningside Drive...,NaN,2,3,USD,No,Thumbnail,...,"$1,40",Monthly,1650,3101 Morningside Dr,Raleigh,NC,358.230,-786.438,RentLingo,1.577360e+09
3,5668639659,housing/rent/apartment,Two BR 209 Aegean Way,"This unit is located at 209 Aegean Way, Vacavi...",NaN,1,2,USD,No,Thumbnail,...,"$1,60",Monthly,820,209 Aegean Way,Vacaville,CA,383.622,-1.219.712,RentLingo,1.577360e+09
4,5668639374,housing/rent/apartment,One BR 4805 Marquette NE,"This unit is located at 4805 Marquette NE, Alb...",NaN,1,1,USD,No,Thumbnail,...,$975,Monthly,624,4805 Marquette NE,Albuquerque,NM,351.038,-1.066.110,RentLingo,1.577360e+09


In [32]:
df.dtypes

id                object
category          object
title             object
body              object
amenities         object
bathrooms         object
bedrooms          object
currency          object
fee               object
has_photo         object
pets_allowed      object
price            float64
price_display     object
price_type        object
square_feet       object
address           object
cityname          object
state             object
latitude         float64
longitude         object
source            object
time             float64
dtype: object

Los campos tipo object representan campos que pueden contener registros nulos o errores de escritura. Se revisará el campo **id** para empezar.
Creamos una función para revisar si el campo **id** es numérico o no.

In [33]:
def check_id(id):
    try:
        int(id)
        return True
    except:
        return False

numericos = df['id'].apply(check_id).sum()
no_numericos = df['id'].shape[0] - numericos
print(f'Numericos: {numericos}, No numéricos: {no_numericos}')

Numericos: 99823, No numéricos: 3


Se observa que hay 3 registros que no son numéricos. Se revisarán estos registros.

In [34]:
df[~df['id'].apply(check_id)]

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
8808,ange of on-site community amenities. Coming so...,Gym,1,1,USD,No,Thumbnail,"Cats,Dogs",1380,"$1,38",...,NaN,Lawrence,NJ,402.659,-746.959,RentDigs.com,1.568782e+09,NaN,NaN,NaN
33780,xpansive rooftop entertainment deck and many p...,"Parking,Patio/Deck",1,1,USD,No,Thumbnail,NaN,2150,"$2,15",...,NaN,Los Angeles,CA,340.432,-1.182.509,RentDigs.com,1.568745e+09,NaN,NaN,NaN
75947,"ellite,Clubhouse,Dishwasher,Garbage Disposal,G...",2,2,USD,No,Thumbnail,"Cats,Dogs",1019,"$1,02",Monthly,...,Henrico,VA,376.249,-774.948,RentDigs.com,1550831790,NaN,NaN,NaN,NaN


Se procederá a eliminar estos registros.

In [35]:
df = df[df['id'].apply(check_id)]

Se revisan registros duplicados.

In [36]:
df[df.duplicated()]

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
42261,5508806580,housing/rent/apartment,"Las Vegas, prime location Two BR, Apartment. P...",Live in the middle of it all! South is a pet f...,"Gym,Parking,Pool",2,2,USD,No,Yes,...,"$1,34",Monthly,1084,NaN,Las Vegas,NV,361.536,-1.151.965,RentDigs.com,1.568754e+09
42262,5508806428,housing/rent/apartment,Northglenn Value. Pet OK!,"Square footage: 738 square ft, unit number: 06...","Gym,Parking,Pool,Storage",1,1,USD,No,Yes,...,"$1,33",Monthly,738,NaN,Northglenn,CO,398.999,-1.049.442,RentDigs.com,1.568754e+09
42263,5508806391,housing/rent/apartment,Apartment in move in condition in District Hei...,Capital Courts has something for everyone. Loc...,"AC,Parking,Playground,Pool",1.5,3,USD,No,Yes,...,"$1,32",Monthly,943,NaN,District Heights,MD,388.516,-768.871,RentDigs.com,1.568754e+09
42264,5508806299,housing/rent/apartment,"District Heights, prime location Two BR, Apart...",Income Requirement: Must have 2. 5x the rent i...,Parking,1,2,USD,No,Yes,...,"$1,35",Monthly,931,NaN,District Heights,MD,388.516,-768.871,RentDigs.com,1.568754e+09
42265,5508806233,housing/rent/apartment,"Spacious One BR, One BA","Square footage: 680 sq-ft, unit number: 122. P...","Basketball,Gym,Patio/Deck,Playground,Pool",1,1,USD,No,Yes,...,"$1,26",Monthly,680,NaN,Denver,CO,398.999,-1.049.442,RentDigs.com,1.568754e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83577,5197839500,housing/rent/apartment,"Apartment, Studio, 400 sq. ft. - in a great area.",Apartment for rent in Ruckersville. Dogs Dogs ...,NaN,1,0,USD,No,No,...,$725,Monthly,400,NaN,Ruckersville,VA,382.203,-783.844,RentDigs.com,1.550824e+09
83578,5197836604,housing/rent/apartment,offers the lifestyle you deserve in the center...,"Our pet friendly one, 2 and 3 beds apartment h...",NaN,2,2,USD,No,Yes,...,"$1,24",Monthly,1051,NaN,Charlotte,NC,353.201,-807.409,RentDigs.com,1.550824e+09
83579,5197834189,housing/rent/apartment,"Attractive Two BR, Two BA. Pet OK!","Square footage: 1055 square feet, unit number:...","Gym,Parking,Pool",2,2,USD,No,Yes,...,"$1,48",Monthly,1055,NaN,Tempe,AZ,333.924,-1.119.265,RentDigs.com,1.550824e+09
83580,5197828852,housing/rent/apartment,The Best of the Best in the City of Tempe! Sav...,In the heart of vibrant Tempe near Arizona Sta...,"Gym,Parking,Pool",2,3,USD,No,Yes,...,"$1,88",Monthly,1261,NaN,Tempe,AZ,333.924,-1.119.265,RentDigs.com,1.550823e+09


Se eliminan los registros duplicados.

In [37]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

Se revisa la columna **category**.

In [38]:
df['category'].value_counts()

category
housing/rent/apartment            99678
housing/rent/commercial/retail       42
housing/rent                          7
housing/rent/short_term               4
housing/rent/home                     4
housing/rent/condo                    3
housing/rent/other                    1
Name: count, dtype: int64

Dado que la mayoría de los registros son de la categoría **Apartments**, se elegirá esta categoría para el análisis.

In [39]:
df = df[df['category'] == 'housing/rent/apartment']

Revisamos la variable **price_type**.

In [40]:
df['price_type'].value_counts()

price_type
Monthly           99671
Weekly                3
Monthly|Weekly        1
Name: count, dtype: int64

Dado que la mayoría de los registros son de tipo **monthly**, se elegirá esta categoría para el análisis.

In [41]:
df = df[df['price_type'] == 'Monthly']

Se analiza la variable **cityname**.

In [42]:
df['cityname'].value_counts()

cityname
Dallas               2860
Denver               2754
Los Angeles          2430
Las Vegas            1849
Arlington            1577
                     ... 
Ponte Vedra Beach       1
Elkins                  1
Keaau                   1
Narberth                1
Clawson                 1
Name: count, Length: 2979, dtype: int64

Dado que la variable **cityname** tiene muchos valores, se omitirá para el análisis de clustering, ya que no se puede considerar como una variable categórica.

In [43]:
df = df.drop(columns=['cityname'])

Se revisa la variable **pets_allowed**.

In [45]:
df['pets_allowed'].value_counts()

pets_allowed
Cats,Dogs         37174
Cats               1845
Dogs                127
Cats,Dogs,None        1
Name: count, dtype: int64

Se convierte la variable **pets_allowed_cat** a tipo categórica.

In [46]:
df['pets_allowed_cat'] = df['pets_allowed'].fillna('No').map({'Cats,Dogs': 'Si', 'Cats': 'Si', 'Dogs': 'Si', 'Cats,Dogs,None': 'Si', 'No': 'No'})
df['pets_allowed_cat'].value_counts()

pets_allowed_cat
No    60524
Si    39147
Name: count, dtype: int64

In [57]:
labelencoder = preprocessing.LabelEncoder()
df['pets_allowed_cat'] = labelencoder.fit_transform(df['pets_allowed_cat'])
df['pets_allowed_cat'].value_counts()

pets_allowed_cat
0    60524
1    39147
Name: count, dtype: int64

### 2. De acuerdo con lo anterior, seleccione las variables que utilizará para realizar la segmentación, y con ello determine el modelo a utilizar.
Se seleccionarán las variables **bathrooms**, **bedrooms**, **pets_allowed_cat**, **price**, **square_feet**, **latitude** y **longitude**.
Se utilizará el modelo K-Prototypes, ya que se tienen variables numéricas y categóricas.
#TODO Falta especificar por que se descartan las variables.

In [51]:
df_vals = df.filter(items=['bathrooms','bedrooms', 'pets_allowed_cat', 'price', 'square_feet', 'latitude', 'longitude'], axis=1)

Revisamos si hay valores nulos.

In [54]:
df_vals.isnull().sum()

bathrooms            61
bedrooms            123
pets_allowed_cat      0
price                 1
square_feet           0
latitude             25
longitude            25
dtype: int64

Se eliminan los registros nulos ya que la cantidad es baja.

In [55]:
df_vals = df_vals.dropna()
df_vals.isnull().sum()

bathrooms           0
bedrooms            0
pets_allowed_cat    0
price               0
square_feet         0
latitude            0
longitude           0
dtype: int64

bathrooms           0
bedrooms            0
pets_allowed_cat    0
price               0
square_feet         0
latitude            0
longitude           0
dtype: int64